<a href="https://colab.research.google.com/github/pratapponnam/ProstateDetection/blob/main/nnunetv1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install nnunet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.6/276.6 kB 8.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.3/156.3 kB 15.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 6.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 87.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 73.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:

In [1]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118


In [2]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install nnunetv2


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.3/206.3 kB 7.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached argparse-1.4.0-py2.py3-none-any.whl.metadata (2.8 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 93.7 MB/s eta 0:00:00
Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)
  Created wheel for nnunetv2: filename=nnunetv2-2.6.0-py3-none-any.whl size=277014 sha256=f28921449215fd221bd00aed34425da9ba03342305d971a48117b2ecf5b8b78f
  Stored in directory: /root/.cache/pip/wheels/3d/0d/bb/f932cb0032d1aacc2f336de421448313d147db8eed671096e9
  Created wheel for ac

In [1]:
import os
import glob
import numpy as np
import nibabel as nib
import pydicom
import tensorflow as tf
import json
import pandas as pd
from tqdm import tqdm


In [2]:

# Paths
BASE_PATH = "/content/drive/MyDrive/"
DATASET_PATH = os.path.join(BASE_PATH, "Prostate")
NIFTI_SAVE_PATH = os.path.join(BASE_PATH, "Prostate_NIfTI_Cache")  # Cache location
NNUNET_RAW_PATH = os.path.join(BASE_PATH, "nnUNet_raw/Dataset500_ProstateX")
IMAGES_TR_PATH = os.path.join(NNUNET_RAW_PATH, "imagesTr")
LABELS_TR_PATH = os.path.join(NNUNET_RAW_PATH, "labelsTr")

# Ensure directories exist
os.makedirs(IMAGES_TR_PATH, exist_ok=True)
os.makedirs(LABELS_TR_PATH, exist_ok=True)
os.makedirs(NIFTI_SAVE_PATH, exist_ok=True)

# Load CSV files
image_train_file = "/content/drive/MyDrive/ProstateX-Images-Train1.csv"
findings_train_file = "/content/drive/MyDrive/ProstateX-Findings-Train1.csv"

if not os.path.exists(image_train_file) or not os.path.exists(findings_train_file):
    print("[ERROR] One or both CSV files are missing. Check the paths.")
    exit(1)

# Load data
image_train_df = pd.read_csv(image_train_file)
findings_train_df = pd.read_csv(findings_train_file)

# Print column names to check
print("[INFO] ImageTrain Columns:", image_train_df.columns)
print("[INFO] FindingsTrain Columns:", findings_train_df.columns)

# Ensure 'ProxID' and 'fid' exist in both CSVs
required_columns = ["ProxID", "fid"]
for col in required_columns:
    if col not in image_train_df.columns or col not in findings_train_df.columns:
        print(f"[ERROR] Column '{col}' not found in both CSV files.")
        exit(1)

# Merge both datasets on 'ProxID' and 'fid'
merged_df = pd.merge(image_train_df, findings_train_df, on=["ProxID", "fid"], how="inner")

print(f"[INFO] Successfully merged! Total records: {len(merged_df)}")

[INFO] ImageTrain Columns: Index(['ProxID', 'Name', 'fid', 'pos', 'WorldMatrix', 'ijk', 'TopLevel',
       'SpacingBetweenSlices', 'VoxelSpacing', 'Dim', 'DCMSerDescr',
       'DCMSerNum'],
      dtype='object')
[INFO] FindingsTrain Columns: Index(['ProxID', 'fid', 'pos', 'zone', 'ClinSig'], dtype='object')
[INFO] Successfully merged! Total records: 187


In [45]:
# Correct column names
SERIES_DESC_COL = "DCMSerDescr"  # Column with Series Description
SERIES_NUM_COL = "DCMSerNum"  # Column with Series Number
LABEL_COL = "ClinSig"  # Column with Label

# Function to load or convert DICOM images
def load_or_convert_dicom_images(dicom_dir, merged_df, img_size=(128, 128)):
    images = []
    labels = []
    matched_records = 0

    dicom_files = glob.glob(os.path.join(dicom_dir, "**/*.dcm"), recursive=True)
    print(f"Found {len(dicom_files)} DICOM files.")

    if len(dicom_files) == 0:
        print("[ERROR] No DICOM files found. Check the directory path.")
        return [], []

    for idx, row in merged_df.iterrows():
        series_desc = row[SERIES_DESC_COL]
        series_num = row[SERIES_NUM_COL]
        label = row[LABEL_COL]

        nifti_file_path = os.path.join(NIFTI_SAVE_PATH, f"prostate_{idx:04d}.nii.gz")

        # If NIfTI already exists, use cached version
        if os.path.exists(nifti_file_path):
            print(f"[INFO] Using cached NIfTI: {nifti_file_path}")
            image = nib.load(nifti_file_path).get_fdata()
            images.append(image)
            labels.append(label)
            matched_records += 1
            continue  # Skip reprocessing

        matched = False
        for dicom_path in dicom_files:
            dicom = pydicom.dcmread(dicom_path)

            if dicom.SeriesNumber == series_num and series_desc in dicom.SeriesDescription:
                img = dicom.pixel_array.astype(np.float32)
                img = (img - np.min(img)) / (np.max(img) - np.min(img))  # Normalize

                # 🔹 Ensure shape is (H, W, 1) before resizing
                if img.ndim == 2:
                    img = np.expand_dims(img, axis=-1)  # Add channel dimension (H, W, 1)

                img = tf.image.resize(img, img_size).numpy()  # Resize to (128, 128, 1)

                # Ensure final shape is (1, 128, 128)
                img = np.transpose(img, (2, 0, 1))  # Convert (128, 128, 1) → (1, 128, 128)

                images.append(img)
                labels.append(label)
                matched_records += 1
                matched = True

                # Save NIfTI in cache with correct spacing and shape
                affine = np.eye(4)  # Default affine matrix
                nifti_img = nib.Nifti1Image(img, affine)
                nifti_img.header.set_zooms((1.0, 1.0, 1.0))  # Ensure correct spacing
                nib.save(nifti_img, nifti_file_path)
                print(f"[INFO] Saved NIfTI: {nifti_file_path}")

                break  # Stop checking once a match is found

        if not matched:
            print(f"[WARNING] No match found for SeriesNumber {series_num} and Description {series_desc}")

    print(f"[INFO] Total matched records: {matched_records}")
    return images, labels

# Convert/load images & labels
images, labels = load_or_convert_dicom_images(DATASET_PATH, merged_df)

Found 9988 DICOM files.
[INFO] Saved NIfTI: /content/drive/MyDrive/Prostate_NIfTI_Cache/prostate_0000.nii.gz
[INFO] Saved NIfTI: /content/drive/MyDrive/Prostate_NIfTI_Cache/prostate_0001.nii.gz
[INFO] Saved NIfTI: /content/drive/MyDrive/Prostate_NIfTI_Cache/prostate_0002.nii.gz
[INFO] Saved NIfTI: /content/drive/MyDrive/Prostate_NIfTI_Cache/prostate_0003.nii.gz
[INFO] Saved NIfTI: /content/drive/MyDrive/Prostate_NIfTI_Cache/prostate_0004.nii.gz
[INFO] Saved NIfTI: /content/drive/MyDrive/Prostate_NIfTI_Cache/prostate_0005.nii.gz
[INFO] Saved NIfTI: /content/drive/MyDrive/Prostate_NIfTI_Cache/prostate_0006.nii.gz
[INFO] Saved NIfTI: /content/drive/MyDrive/Prostate_NIfTI_Cache/prostate_0007.nii.gz
[INFO] Saved NIfTI: /content/drive/MyDrive/Prostate_NIfTI_Cache/prostate_0008.nii.gz
[INFO] Saved NIfTI: /content/drive/MyDrive/Prostate_NIfTI_Cache/prostate_0009.nii.gz
[INFO] Saved NIfTI: /content/drive/MyDrive/Prostate_NIfTI_Cache/prostate_0010.nii.gz
[INFO] Saved NIfTI: /content/drive/MyDriv

In [46]:

# Save images & labels as NIfTI in nnUNet format
for idx, (image, label) in enumerate(tqdm(zip(images, labels), desc="Saving NIfTI files", total=len(images))):
    affine = np.eye(4)  # Default affine matrix

    # Ensure image is in (1, 128, 128) format
    if image.ndim == 2:
        print(f"⚠️ Image {idx} is 2D. Expanding to 3D...")
        image = np.expand_dims(image, axis=0)  # Convert (H, W) → (1, H, W)

    image_nifti = nib.Nifti1Image(image, affine)
    image_nifti.header.set_zooms((1.0, 1.0, 1.0))  # Set correct spacing
    nib.save(image_nifti, os.path.join(IMAGES_TR_PATH, f"prostate_{idx:04d}_0000.nii.gz"))

    # Convert `label` to an integer and ensure shape (1, 128, 128)
    label_array = np.full((1, 128, 128), int(label), dtype=np.int16)  # Create a full array instead of (1,1)

    label_nifti = nib.Nifti1Image(label_array, affine)
    label_nifti.header.set_zooms((1.0, 1.0, 1.0))  # Match spacing to images
    nib.save(label_nifti, os.path.join(LABELS_TR_PATH, f"prostate_{idx:04d}.nii.gz"))

    print(f"[INFO] Saved Image: prostate_{idx:04d}_0000.nii.gz, Label: prostate_{idx:04d}.nii.gz")


Saving NIfTI files:   3%|▎         | 5/187 [00:00<00:03, 46.00it/s]

[INFO] Saved Image: prostate_0000_0000.nii.gz, Label: prostate_0000.nii.gz
[INFO] Saved Image: prostate_0001_0000.nii.gz, Label: prostate_0001.nii.gz
[INFO] Saved Image: prostate_0002_0000.nii.gz, Label: prostate_0002.nii.gz
[INFO] Saved Image: prostate_0003_0000.nii.gz, Label: prostate_0003.nii.gz
[INFO] Saved Image: prostate_0004_0000.nii.gz, Label: prostate_0004.nii.gz
[INFO] Saved Image: prostate_0005_0000.nii.gz, Label: prostate_0005.nii.gz
[INFO] Saved Image: prostate_0006_0000.nii.gz, Label: prostate_0006.nii.gz
[INFO] Saved Image: prostate_0007_0000.nii.gz, Label: prostate_0007.nii.gz
[INFO] Saved Image: prostate_0008_0000.nii.gz, Label: prostate_0008.nii.gz


Saving NIfTI files:   8%|▊         | 15/187 [00:00<00:04, 40.68it/s]

[INFO] Saved Image: prostate_0009_0000.nii.gz, Label: prostate_0009.nii.gz
[INFO] Saved Image: prostate_0010_0000.nii.gz, Label: prostate_0010.nii.gz
[INFO] Saved Image: prostate_0011_0000.nii.gz, Label: prostate_0011.nii.gz
[INFO] Saved Image: prostate_0012_0000.nii.gz, Label: prostate_0012.nii.gz
[INFO] Saved Image: prostate_0013_0000.nii.gz, Label: prostate_0013.nii.gz
[INFO] Saved Image: prostate_0014_0000.nii.gz, Label: prostate_0014.nii.gz
[INFO] Saved Image: prostate_0015_0000.nii.gz, Label: prostate_0015.nii.gz
[INFO] Saved Image: prostate_0016_0000.nii.gz, Label: prostate_0016.nii.gz
[INFO] Saved Image: prostate_0017_0000.nii.gz, Label: prostate_0017.nii.gz
[INFO] Saved Image: prostate_0018_0000.nii.gz, Label: prostate_0018.nii.gz


Saving NIfTI files:  14%|█▍        | 26/187 [00:00<00:03, 44.30it/s]

...

[INFO] Saved Image: prostate_0169_0000.nii.gz, Label: prostate_0169.nii.gz
[INFO] Saved Image: prostate_0170_0000.nii.gz, Label: prostate_0170.nii.gz
[INFO] Saved Image: prostate_0171_0000.nii.gz, Label: prostate_0171.nii.gz
[INFO] Saved Image: prostate_0172_0000.nii.gz, Label: prostate_0172.nii.gz
[INFO] Saved Image: prostate_0173_0000.nii.gz, Label: prostate_0173.nii.gz
[INFO] Saved Image: prostate_0174_0000.nii.gz, Label: prostate_0174.nii.gz
[INFO] Saved Image: prostate_0175_0000.nii.gz, Label: prostate_0175.nii.gz
[INFO] Saved Image: prostate_0176_0000.nii.gz, Label: prostate_0176.nii.gz


Saving NIfTI files:  98%|█████████▊| 184/187 [00:04<00:00, 40.59it/s]

[INFO] Saved Image: prostate_0177_0000.nii.gz, Label: prostate_0177.nii.gz
[INFO] Saved Image: prostate_0178_0000.nii.gz, Label: prostate_0178.nii.gz
[INFO] Saved Image: prostate_0179_0000.nii.gz, Label: prostate_0179.nii.gz
[INFO] Saved Image: prostate_0180_0000.nii.gz, Label: prostate_0180.nii.gz
[INFO] Saved Image: prostate_0181_0000.nii.gz, Label: prostate_0181.nii.gz
[INFO] Saved Image: prostate_0182_0000.nii.gz, Label: prostate_0182.nii.gz
[INFO] Saved Image: prostate_0183_0000.nii.gz, Label: prostate_0183.nii.gz
[INFO] Saved Image: prostate_0184_0000.nii.gz, Label: prostate_0184.nii.gz
[INFO] Saved Image: prostate_0185_0000.nii.gz, Label: prostate_0185.nii.gz


Saving NIfTI files: 100%|██████████| 187/187 [00:04<00:00, 42.61it/s]

[INFO] Saved Image: prostate_0186_0000.nii.gz, Label: prostate_0186.nii.gz


In [5]:
import os
import json
import subprocess

# Define paths
BASE_PATH = "/content/drive/MyDrive/nnUNet_raw"
DATASET_PATH = os.path.join(BASE_PATH, "Dataset500_ProstateX")
IMAGES_TR_PATH = os.path.join(DATASET_PATH, "imagesTr")
LABELS_TR_PATH = os.path.join(DATASET_PATH, "labelsTr")
PREPROCESSED_PATH = "/content/drive/MyDrive/nnUNet_preprocessed"
RESULTS_PATH = "/content/drive/MyDrive/nnUNet_results"
PREDICTIONS_PATH = "/content/drive/MyDrive/nnUNet_predictions"

# Ensure dataset is in the correct location
if not os.path.exists(DATASET_PATH):
    print(f"[ERROR] Dataset path {DATASET_PATH} does not exist! Ensure it is placed correctly inside nnUNet_raw.")
    exit(1)

# Set nnUNet environment variables
os.environ["nnUNet_raw"] = BASE_PATH
os.environ["nnUNet_preprocessed"] = PREPROCESSED_PATH
os.environ["nnUNet_results"] = RESULTS_PATH

# Verify environment variables
print("[INFO] Environment Variables Set:")
for var in ["nnUNet_raw", "nnUNet_preprocessed", "nnUNet_results"]:
    print(f"{var}: {os.environ.get(var)}")

# List available images for dataset.json
try:
    images = sorted([f for f in os.listdir(IMAGES_TR_PATH) if f.endswith(".nii.gz")])
    print(f"[INFO] Found {len(images)} training images.")
except FileNotFoundError:
    print("[ERROR] Training image directory not found!")
    exit(1)


[INFO] Environment Variables Set:
nnUNet_raw: /content/drive/MyDrive/nnUNet_raw
nnUNet_preprocessed: /content/drive/MyDrive/nnUNet_preprocessed
nnUNet_results: /content/drive/MyDrive/nnUNet_results
[INFO] Found 187 training images.


In [48]:


# Generate dataset.json with required keys
dataset_json = {
    "name": "ProstateX",
    "description": "ProstateX dataset for nnU-Net training",
    "tensorImageSize": "3D",
    "reference": "",
    "licence": "",
    "release": "1.0",
    "modality": {"0": "MRI"},
    "labels": { "background":0
               , "tumor":1},
    "channel_names": {"0": "T2"},  # Adjust based on your data
    "file_ending": ".nii.gz",
    "numTraining": len(images),
    "numTest": 0,
    "training": [{"image": f"./imagesTr/{img}", "label": f"./labelsTr/{img.replace('_0000', '')}"} for img in images],
    "test": []
}

# Save dataset.json
json_path = os.path.join(DATASET_PATH, "dataset.json")
with open(json_path, "w") as f:
    json.dump(dataset_json, f, indent=4)

print("[SUCCESS] dataset.json created successfully!")

# Verify dataset.json content
with open(json_path, "r") as f:
    data = json.load(f)

required_keys = ["labels", "channel_names", "numTraining", "file_ending"]
missing_keys = [key for key in required_keys if key not in data]

if not missing_keys:
    print("[SUCCESS] dataset.json is correctly formatted!")
else:
    print(f"[ERROR] Missing keys: {missing_keys}")
    exit(1)

[SUCCESS] dataset.json created successfully!
[SUCCESS] dataset.json is correctly formatted!


In [49]:
# Step 4: Preprocess the Data
print("[INFO] Running nnUNet Preprocessing...")
preprocess_command = "nnUNetv2_plan_and_preprocess -d 500 --verify_dataset_integrity"
process = subprocess.run(preprocess_command, shell=True, capture_output=True, text=True)
print("[INFO] Preprocessing Output:\n", process.stdout)
if process.stderr:
    print("[ERROR] Preprocessing Error:\n", process.stderr)
    exit(1)

[INFO] Running nnUNet Preprocessing...
[INFO] Preprocessing Output:
 Fingerprint extraction...
Dataset500_ProstateX
Using <class 'nnunetv2.imageio.simpleitk_reader_writer.SimpleITKIO'> as reader/writer

####################
verify_dataset_integrity Done. 
If you didn't see any error messages then your dataset is most likely OK!
####################

Using <class 'nnunetv2.imageio.simpleitk_reader_writer.SimpleITKIO'> as reader/writer
Experiment planning...

############################
INFO: You are using the old nnU-Net default planner. We have updated our recommendations. Please consider using those instead! Read more here: https://github.com/MIC-DKFZ/nnUNet/blob/master/documentation/resenc_presets.md
############################

Dropping 3d_lowres config because the image size difference to 3d_fullres is too small. 3d_fullres: [126. 126.   1.], 3d_lowres: [126, 126, 1]
2D U-Net configuration:
{'data_identifier': 'nnUNetPlans_2d', 'preprocessor_name': 'DefaultPreprocessor', 'batch_s

In [52]:
!apt-get update -y
!apt-get install -y python3-dev python3-pip python3-venv
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu118


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,767 kB]
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [3,813 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/universe amd64

In [1]:
import torch
print("CUDA Available:", torch.cuda.is_available())

CUDA Available: True


In [ ]:
import os
import json
import subprocess

# Step 5: Train the Model (Fixed missing fold argument)
print("[INFO] Training nnUNet Model...")
train_command = "nnUNetv2_train 500 3d_fullres 0"
process = subprocess.run(train_command, shell=True, capture_output=True, text=True)
print("[INFO] Training Output:\n", process.stdout)
if process.stderr:
    print("[ERROR] Training Error:\n", process.stderr)
    exit(1)



[INFO] Training nnUNet Model...


In [ ]:

# Step 6: Inference (Fixed dataset path)
print("[INFO] Running Inference...")
predict_command = f"nnUNetv2_predict -i {IMAGES_TR_PATH} -o {PREDICTIONS_PATH} -d 500 -c 3d_fullres -f 0"
process = subprocess.run(predict_command, shell=True, capture_output=True, text=True)
print("[INFO] Inference Output:\n", process.stdout)
if process.stderr:
    print("[ERROR] Inference Error:\n", process.stderr)
    exit(1)

print("[INFO] nnUNet Workflow Completed Successfully!")

In [ ]:
import os

os.environ["nnUNet_raw"] = "/content/drive/MyDrive/nnUNet_raw"
os.environ["nnUNet_preprocessed"] = "/content/drive/MyDrive/nnUNet_preprocessed"
os.environ["nnUNet_results"] = "/content/drive/MyDrive/nnUNet_results"

# Verify environment variables
print("[INFO] Environment Variables Set:")
print("nnUNet_raw:", os.environ.get("nnUNet_raw"))
print("nnUNet_preprocessed:", os.environ.get("nnUNet_preprocessed"))
print("nnUNet_results:", os.environ.get("nnUNet_results"))

[INFO] Environment Variables Set:
nnUNet_raw: /content/drive/MyDrive/nnUNet_raw
nnUNet_preprocessed: /content/drive/MyDrive/nnUNet_preprocessed
nnUNet_results: /content/drive/MyDrive/nnUNet_results


In [ ]:
import subprocess

# Define paths
IMAGES_TR_PATH = "/content/drive/MyDrive/nnUNet_raw/Task500_ProstateX/imagesTr"
PREDICTION_PATH = "/content/drive/MyDrive/nnUNet_predictions"

# Ensure output directory exists
os.makedirs(PREDICTION_PATH, exist_ok=True)

# Run inference command and capture errors
inference_command = f"nnUNetv2_predict -i {IMAGES_TR_PATH} -o {PREDICTION_PATH} -d 500 -c 3d_fullres"
process = subprocess.run(inference_command, shell=True, capture_output=True, text=True)

# Print the command output
print("[INFO] Command Output:\n", process.stdout)
print("[ERROR] Command Error:\n", process.stderr)

# Check for failure
if process.returncode != 0:
    print(f"[ERROR] Inference failed with exit code {process.returncode}. Check the logs above.")


[INFO] Command Output:
 
#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#######################################################################


[ERROR] Command Error:
 Traceback (most recent call last):
  File "/usr/local/bin/nnUNetv2_predict", line 8, in <module>
    sys.exit(predict_entry_point())
             ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nnunetv2/inference/predict_from_raw_data.py", line 948, in predict_entry_point
    model_folder = get_output_folder(args.d, args.tr, args.p, args.c)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nnunetv2/utilities/file_path_utilities.py", line 22, in get_outpu

In [ ]:
DATASET_ID = "002"
inference_command = f"nnUNetv2_predict -i {IMAGES_TR_PATH} -o {PREDICTION_PATH} -d {DATASET_ID} -c 3d_fullres"


In [ ]:
import subprocess

# Run inference command
inference_command = f"nnUNetv2_predict -i {IMAGES_TR_PATH} -o {PREDICTION_PATH} -d 500 -c 3d_fullres"
process = subprocess.run(inference_command, shell=True, capture_output=True, text=True)

# Print output and error
print("[INFO] Command Output:\n", process.stdout)
print("[ERROR] Command Error:\n", process.stderr)


[INFO] Command Output:
 
#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#######################################################################


[ERROR] Command Error:
 Traceback (most recent call last):
  File "/usr/local/bin/nnUNetv2_predict", line 8, in <module>
    sys.exit(predict_entry_point())
             ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nnunetv2/inference/predict_from_raw_data.py", line 948, in predict_entry_point
    model_folder = get_output_folder(args.d, args.tr, args.p, args.c)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nnunetv2/utilities/file_path_utilities.py", line 22, in get_outpu

In [ ]:
import os

nnUNet_preprocessed_path = "/content/drive/MyDrive/nnUNet_preprocessed"
nnUNet_results_path = "/content/drive/MyDrive/nnUNet_results"

# List available dataset folders
print("[INFO] Checking available datasets in nnUNet_preprocessed:")
print(os.listdir(nnUNet_preprocessed_path))

print("\n[INFO] Checking available trained models in nnUNet_results:")
print(os.listdir(nnUNet_results_path))


[INFO] Checking available datasets in nnUNet_preprocessed:


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/nnUNet_preprocessed'

In [ ]:
import os

# Define paths
base_path = "/content/drive/MyDrive/"
paths = ["nnUNet_raw", "nnUNet_preprocessed", "nnUNet_results"]

for path in paths:
    full_path = os.path.join(base_path, path)
    print(f"[INFO] Checking path: {full_path}")
    print("Exists:", os.path.exists(full_path))
    print("Contents:", os.listdir(full_path) if os.path.exists(full_path) else "Folder not found")
    print("\n")



[INFO] Checking path: /content/drive/MyDrive/nnUNet_raw
Exists: True
Contents: ['Task500_ProstateX']


[INFO] Checking path: /content/drive/MyDrive/nnUNet_preprocessed
Exists: False
Contents: Folder not found


[INFO] Checking path: /content/drive/MyDrive/nnUNet_results
Exists: False
Contents: Folder not found




In [ ]:
nnUNetv2_plan_and_preprocess -d 500 --verify_dataset_integrity


SyntaxError: invalid syntax (<ipython-input-22-5e82b98bdce9>, line 1)

In [ ]:
import os
import shutil

# Define paths
TRAINED_MODEL_PATH = "/content/drive/MyDrive/nnUNet_results/nnUNet/3d_fullres/Task500_ProstateX"
BACKUP_MODEL_PATH = "/content/drive/MyDrive/nnUNet_trained_model_backup/Task500_ProstateX"

# Ensure backup directory exists
os.makedirs(os.path.dirname(BACKUP_MODEL_PATH), exist_ok=True)

# Copy the trained model to backup
if os.path.exists(TRAINED_MODEL_PATH):
    shutil.copytree(TRAINED_MODEL_PATH, BACKUP_MODEL_PATH, dirs_exist_ok=True)
    print(f"[INFO] Trained model successfully backed up to {BACKUP_MODEL_PATH}")
else:
    print("[ERROR] Trained model not found. Ensure training was completed successfully.")


[ERROR] Trained model not found. Ensure training was completed successfully.


In [ ]:
import os
import subprocess

# Define paths
os.environ["nnUNet_raw"] = "/content/drive/MyDrive/nnUNet_raw"
os.environ["nnUNet_preprocessed"] = "/content/drive/MyDrive/nnUNet_preprocessed"
os.environ["nnUNet_results"] = "/content/drive/MyDrive/nnUNet_results"

# Verify environment variables
print("[INFO] Environment Variables Set:")
print("nnUNet_raw:", os.environ.get("nnUNet_raw"))
print("nnUNet_preprocessed:", os.environ.get("nnUNet_preprocessed"))
print("nnUNet_results:", os.environ.get("nnUNet_results"))

# Run nnU-Net Preprocessing
print("[INFO] Running nnU-Net Preprocessing...")
preprocess_command = "nnUNetv2_plan_and_preprocess -d 500 --verify_dataset_integrity"
process = subprocess.run(preprocess_command, shell=True, capture_output=True, text=True)

# Print command output
print("[INFO] Command Output:\n", process.stdout)
print("[ERROR] Command Error:\n", process.stderr)

# Verify if preprocessing was successful
preprocessed_path = "/content/drive/MyDrive/nnUNet_preprocessed/Task500_ProstateX"
if os.path.exists(preprocessed_path):
    print(f"[SUCCESS] Preprocessing completed! Preprocessed data is in: {preprocessed_path}")
    print("Listing preprocessed files:")
    print(os.listdir(preprocessed_path))
else:
    print("[ERROR] Preprocessing failed. Check the error logs above.")


[INFO] Environment Variables Set:
nnUNet_raw: /content/drive/MyDrive/nnUNet_raw
nnUNet_preprocessed: /content/drive/MyDrive/nnUNet_preprocessed
nnUNet_results: /content/drive/MyDrive/nnUNet_results
[INFO] Running nnU-Net Preprocessing...
[INFO] Command Output:
 Fingerprint extraction...
Dataset500_ProstateX

[ERROR] Command Error:
 Traceback (most recent call last):
  File "/usr/local/bin/nnUNetv2_plan_and_preprocess", line 8, in <module>
    sys.exit(plan_and_preprocess_entry())
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nnunetv2/experiment_planning/plan_and_preprocess_entrypoints.py", line 180, in plan_and_preprocess_entry
    extract_fingerprints(args.d, args.fpe, args.npfp, args.verify_dataset_integrity, args.clean, args.verbose)
  File "/usr/local/lib/python3.11/dist-packages/nnunetv2/experiment_planning/plan_and_preprocess_api.py", line 47, in extract_fingerprints
    extract_fingerprint_dataset(d, fingerprint_extractor_class, num_pro

In [ ]:
import os

dataset_path = "/content/drive/MyDrive/nnUNet_raw/Dataset500_ProstateX"
print("Dataset exists:", os.path.exists(dataset_path))
print("Contents:", os.listdir(dataset_path) if os.path.exists(dataset_path) else "Dataset folder missing!")


Dataset exists: True
Contents: ['imagesTr', 'labelsTr', 'dataset.json']


In [ ]:
import os

nnunet_raw_path = "/content/drive/MyDrive/nnUNet_raw"
if os.path.exists(nnunet_raw_path):
    print("nnUNet_raw contents:", os.listdir(nnunet_raw_path))
else:
    print("nnUNet_raw folder is missing!")


nnUNet_raw contents: ['Dataset500_ProstateX']


In [ ]:
import os

old_path = "/content/drive/MyDrive/nnUNet_raw/Task500_ProstateX"
new_path = "/content/drive/MyDrive/nnUNet_raw/Dataset500_ProstateX"

# Check if the old path exists before renaming
if os.path.exists(old_path):
    os.rename(old_path, new_path)
    print(f"[SUCCESS] Renamed '{old_path}' to '{new_path}'")
else:
    print("[ERROR] Task500_ProstateX not found!")


[ERROR] Task500_ProstateX not found!
